# Bilateral Symmetry

## Params to change

For bilateral_symm: 

sliding_window_len = 120
rolling_mean_smoothing_window = 5
savgol_window_len = 70
savgol_polyorder = 3
signal = "trajectory"

For cca:

sliding_window_len = 120
rolling_mean_smoothing_window = 5
savgol_window_len = 70
savgol_polyorder = 3
signal = "velocity"

In [1]:
num_file = 1
sliding_window_len = 120
rolling_mean_smoothing_window = 5
savgol_window_len = 70
savgol_polyorder = 3


feature_to_plot = 'bilateral_symmetry_index'
signal = "trajectory" # or "trajectory"

In [2]:
import pandas as pd
from pathlib import Path

bilateral_dir = Path().cwd()
txt_files = sorted(bilateral_dir.glob("*.txt"))
df = pd.read_csv(txt_files[num_file], skiprows=4, sep="\t")
list(zip(range(len(df.columns)),df.columns))
cols_to_keep = [col for col in df.columns if (col.endswith("x") or col.endswith("y") or col.endswith("z")) and not "speed" in col and not "dist" in col][2:]
df = df[cols_to_keep].drop_duplicates()
df.head()

,Elbowloutx,Elbowlouty,Elbowloutz,Elbowroutx,Elbowrouty,Elbowroutz,Hipoutx,Hipouty,Hipoutz,Kneesoutx,...,Neckoutz,Rightwristoutx,Rightwristouty,Rightwristoutz,Shoulderloutx,Shoulderlouty,Shoulderloutz,Shoulderroutx,Shoulderrouty,Shoulderroutz
0,0.133800,-0.595536,3.247540,0.599985,-0.465516,3.077977,0.381360,-0.898628,3.052152,0.365476,...,3.337072,0.545064,-0.483865,2.873700,0.213274,-0.412063,3.286905,0.547330,-0.381526,3.269114
1,0.134663,-0.593051,3.245115,0.592586,-0.476171,3.043021,0.381811,-0.898640,3.052119,0.366608,...,3.333560,0.540887,-0.489659,2.837706,0.212316,-0.413334,3.286154,0.544998,-0.378322,3.264117
3,0.135136,-0.591851,3.243853,0.644535,-0.493934,3.060604,0.379470,-0.906583,3.056382,0.368128,...,3.326908,0.589576,-0.486210,2.856295,0.212659,-0.412931,3.285850,0.543259,-0.375462,3.258880
5,0.137144,-0.582053,3.242322,0.649305,-0.488310,3.084353,0.380127,-0.908115,3.058009,0.370024,...,3.315791,0.426740,-0.468891,2.919812,0.211540,-0.412369,3.285370,0.539133,-0.365768,3.241401
7,0.136090,-0.581701,3.241711,0.617727,-0.473326,3.008617,0.379406,-0.895127,3.037995,0.372685,...,3.302498,0.391134,-0.467487,2.906243,0.210633,-0.410495,3.284431,0.532908,-0.348345,3.200350


In [3]:
import numpy as np
print("Before cleaning:")
if df.isna().any(axis=None) or (df == 0.0).any(axis=None):
    print("\033[91mThe dataframe contains NaN or zero values.\033[0m")
    
df.replace(0, np.nan, inplace=True)
df.interpolate(method='linear', inplace=True)
df.ffill(inplace=True)
df.bfill(inplace=True)  # Backfill to handle any remaining NaNs

print("After cleaning:")
if df.isna().any(axis=None) or (df == 0.0).any(axis=None):
    print("\033[91mWARNING: The dataframe contains NaN or zero values.\033[0m")
else:
    print("\033[92mThe dataframe does not contain NaN or zero values.\033[0m")

Before cleaning:
The dataframe contains NaN or zero values.
After cleaning:
The dataframe does not contain NaN or zero values.


In [4]:
markersPosTableX = df.iloc[:,0::3]
markersPosTableZ = df.iloc[:,1::3]
markersPosTableY = df.iloc[:,2::3]


markersPosTableX = markersPosTableX.rename(columns=dict(zip(list(markersPosTableX),[name.replace("out","")[:-1]+"x" for name in list(markersPosTableX)])))
markersPosTableY = markersPosTableY.rename(columns=dict(zip(list(markersPosTableY),[name.replace("out","")[:-1]+"y" for name in list(markersPosTableY)])))
markersPosTableZ = markersPosTableZ.rename(columns=dict(zip(list(markersPosTableZ),[name.replace("out","")[:-1]+"z" for name in list(markersPosTableZ)])))
markersPosTableX = markersPosTableX.reindex(sorted(markersPosTableX.columns), axis=1)
markersPosTableY = markersPosTableY.reindex(sorted(markersPosTableY.columns), axis=1)
markersPosTableZ = markersPosTableZ.reindex(sorted(markersPosTableZ.columns), axis=1)

print(markersPosTableX.head())
print(markersPosTableY.head())
print(markersPosTableZ.head())
markers_names = [col[:-1] for col in markersPosTableX.columns]
print("Markers names:", markers_names)
colors = ["red"]*2+["black"]*2+["green"]+["black"]+["green"]+["pink"]*2

markersPosX_no_smooth = markersPosTableX.values
markersPosY_no_smooth = markersPosTableY.values
markersPosZ_no_smooth = markersPosTableZ.values

from scipy.signal import savgol_filter

# Apply smoothing to the marker positions
markersPosX = savgol_filter(markersPosTableX.rolling(window=rolling_mean_smoothing_window, min_periods=1, center=True).mean().values, window_length=savgol_window_len, polyorder=savgol_polyorder, axis=0)
markersPosY = savgol_filter(markersPosTableY.rolling(window=rolling_mean_smoothing_window, min_periods=1, center=True).mean().values, window_length=savgol_window_len, polyorder=savgol_polyorder, axis=0)
markersPosZ = savgol_filter(markersPosTableZ.rolling(window=rolling_mean_smoothing_window, min_periods=1, center=True).mean().values, window_length=savgol_window_len, polyorder=savgol_polyorder, axis=0)

if signal == "velocity":
    # Calculate velocity by taking the first derivative of position
    signalX = savgol_filter(np.diff(markersPosX, axis=0), window_length=savgol_window_len//2, polyorder=savgol_polyorder, axis=0)
    signalY = savgol_filter(np.diff(markersPosY, axis=0), window_length=savgol_window_len//2, polyorder=savgol_polyorder, axis=0)
    signalZ = savgol_filter(np.diff(markersPosZ, axis=0), window_length=savgol_window_len//2, polyorder=savgol_polyorder, axis=0)
else:
    signalX, signalY, signalZ = markersPosX, markersPosY, markersPosZ

# Calculate the center of mass for each frame
center_of_mass = np.array([markersPosX.mean(axis=1), markersPosY.mean(axis=1), markersPosZ.mean(axis=1)]).T

    Elbowlx   Elbowrx      Hipx    Kneesx  Leftwristx     Neckx  Rightwristx  \
0  0.133800  0.599985  0.381360  0.365476    0.058858  0.363936     0.545064   
1  0.134663  0.592586  0.381811  0.366608    0.058832  0.364007     0.540887   
3  0.135136  0.644535  0.379470  0.368128    0.060970  0.362082     0.589576   
5  0.137144  0.649305  0.380127  0.370024    0.063947  0.365487     0.426740   
7  0.136090  0.617727  0.379406  0.372685    0.064464  0.367069     0.391134   

   Shoulderlx  Shoulderrx  
0    0.213274    0.547330  
1    0.212316    0.544998  
3    0.212659    0.543259  
5    0.211540    0.539133  
7    0.210633    0.532908  
    Elbowly   Elbowry      Hipy    Kneesy  Leftwristy     Necky  Rightwristy  \
0  3.247540  3.077977  3.052152  2.591405    3.086320  3.337072     2.873700   
1  3.245115  3.043021  3.052119  2.591993    3.084258  3.333560     2.837706   
3  3.243853  3.060604  3.056382  2.592129    3.080072  3.326908     2.856295   
5  3.242322  3.084353  3.058009

In [5]:
import numpy as np

def get_adjacency_matrix(numMarkers, bones):
    edges = np.array(bones)
    adjacencyMatrix = np.zeros((numMarkers,numMarkers),dtype=bool)
    adjacencyMatrix[edges[:,0],edges[:,1]] = True
    return adjacencyMatrix

In [6]:

import tqdm
from pyeyesweb.analysis_primitives.geometric_symmetry import GeometricSymmetry
from pyeyesweb.data_models.sliding_window import SlidingWindow

# Calculate the bilateral symmetry of the two hands using the BilateralSymmetry
sw = SlidingWindow(max_length=sliding_window_len, n_dimensions=3, m_joints=markersPosX.shape[1])
joint_pairs = [(0,1), (4,6), (7,8)]
geometric_symm = GeometricSymmetry(joint_pairs=joint_pairs, center_of_symmetry=5)

values = {(left, right) : {
    'bilateral_symmetry_index': np.zeros(markersPosX.shape[0]),
    'cca_correlation': np.zeros(markersPosX.shape[0])
} for left, right in joint_pairs}

for frame in tqdm.tqdm(range(signalX.shape[0])):
    val = np.array(list(zip(signalX[frame], signalY[frame], signalZ[frame])))
    print(val.shape)
    sw.append(val)
    raise Exception("STOP")
    res = geometric_symm(sw)
    for (left, right), metrics in res.items():
        values[(left, right)]['bilateral_symmetry_index'][frame] = metrics
        #values[(left, right)]['cca_correlation'][frame] = metrics['cca_correlation']

  0%|          | 0/4510 [00:00<?, ?it/s]

(9, 3)
(9, 3)


Exception: Debug: Check the shape of the input value after reshaping.

In [ ]:
values_to_plot = [values[(7,8)][feature_to_plot], values[(0,1)][feature_to_plot], values[(4,6)][feature_to_plot]]
colors_to_plot = [colors[7], colors[0], colors[4]]
joints_names_to_plot = [(markers_names[7], markers_names[8]), (markers_names[0], markers_names[1]), (markers_names[4], markers_names[6])]

In [26]:
print(values)

{(0, 1): {'bilateral_symmetry_index': array([0.7846712 , 0.78097501, 0.77655821, ..., 0.95780952, 0.95791163,
       0.95799571]), 'cca_correlation': array([0., 0., 0., ..., 0., 0., 0.])}, (4, 6): {'bilateral_symmetry_index': array([0.70689544, 0.70276171, 0.69836211, ..., 0.93024901, 0.93022264,
       0.93014316]), 'cca_correlation': array([0., 0., 0., ..., 0., 0., 0.])}, (7, 8): {'bilateral_symmetry_index': array([0.92328784, 0.92339389, 0.92277792, ..., 0.96452342, 0.96450306,
       0.96448712]), 'cca_correlation': array([0., 0., 0., ..., 0., 0., 0.])}}


In [27]:
%matplotlib qt

from matplotlib import pyplot as plt
from matplotlib.widgets import Slider
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from matplotlib.widgets import Button
import matplotlib.animation as animation
#bones = infer_bones(df)
import os

import json
import time

#bones = infer_bones(df)
#print(bones)

if 'anim' in globals() and anim.event_source != None:
    anim.pause()

def get_adjacency_matrix(numMarkers, bones):
    edges = np.array(bones)
    adjacencyMatrix = np.zeros((numMarkers,numMarkers),dtype=bool)
    if bones == []:
        return adjacencyMatrix
    adjacencyMatrix[edges[:,0],edges[:,1]] = True
    return adjacencyMatrix

# Convert bones into adjacency matrix
bones = [[4,0],[0,7],[7,8],[8,1],[1,6],[5,2],[2,3]]
adjacencyMatrix = get_adjacency_matrix(markersPosX.shape[1], bones)

fig = plt.figure(f"3D Movement file:{num_file} feature:{feature_to_plot} signal:{signal}", figsize=(19.20, 10.80))
plt.clf()

# Add a 3D subplot with the specified position and size
ax = fig.add_axes([0, -0.16, 1, .85], projection='3d')
ax.view_init(elev=0, azim=-90)
# Create three stacked subplots for the synchronization plots
left, bottom, width, total_height = 0.05, 0.55, 0.9, 0.42
gap = 0.04
height = (total_height - 2 * gap) / 3

ax_acc1 = fig.add_axes([left, bottom + 2 * (height + gap), width, height])
ax_acc2 = fig.add_axes([left, bottom + (height + gap), width, height])
ax_acc3 = fig.add_axes([left, bottom, width, height])


ax_acc1.plot(values_to_plot[0], color=colors_to_plot[0])
ax_acc2.plot(values_to_plot[1], color=colors_to_plot[1])
ax_acc3.plot(values_to_plot[2], color=colors_to_plot[2])

ax_acc1.legend([f"{joints_names_to_plot[0][0]}-{joints_names_to_plot[0][1]}"], loc='best')
ax_acc2.legend([f"{joints_names_to_plot[1][0]}-{joints_names_to_plot[1][1]}"], loc='best')
ax_acc3.legend([f"{joints_names_to_plot[2][0]}-{joints_names_to_plot[2][1]}"], loc='best')
ax_acc1.set_yticks([0, 0.7, 1])
ax_acc2.set_yticks([0, 0.7, 1])
ax_acc3.set_yticks([0, 0.7, 1])
ax_acc1.set_xticks(list(range(0, len(values_to_plot[2]), 300)))
ax_acc1.set_xticklabels([])  # Hide x-axis ticks for the first two subplots
ax_acc2.set_xticks(list(range(0, len(values_to_plot[2]), 300)))
ax_acc2.set_xticklabels([])  # Hide x-axis ticks for the first two subplots
ax_acc3.set_xticks(list(range(0, len(values_to_plot[2]), 300)))  # Show x-axis ticks for the last subplot
ax_acc3.set_ylim(-0.05, 1.05)
ax_acc1.set_ylim(-0.05, 1.05)
ax_acc2.set_ylim(-0.05, 1.05)
ax_acc3.set_ylim(-0.05, 1.05)
#ax_acc.plot(speed, color='green')
#ax_acc.set_yticks(np.linspace(0,max(acceleration),10).tolist()+ [max(acceleration)+0.05, max(acceleration)+0.55])
#ax_acc.set_yticklabels(np.round(np.linspace(0,max(acceleration),10),3).tolist()+ ["v = 0", f"v = {max(speed):.2f}"])
ax_acc1.set_title(f'Feature: {feature_to_plot} | Signal: {signal} | Sliding window len = {sliding_window_len}', fontsize=10)
ax_acc3.set_xlabel('F r a m e')
#ax_acc.grid(axis='y', color='gray')
ax_acc2.set_ylabel('Symmetry Index')

#ax_acc.plot(peaks, acceleration[peaks], 'ro', markersize=5, label='Peaks')
ax_acc1.grid(axis='both', color='gray')
ax_acc2.grid(axis='both', color='gray')
ax_acc3.grid(axis='both', color='gray')

cursor1 = ax_acc1.plot(0, values_to_plot[0][0], 'bo', markersize=4, label='Current Frame')
cursor2 = ax_acc2.plot(0, values_to_plot[1][0], 'bo', markersize=4, label='Current Frame')
cursor3 = ax_acc3.plot(0, values_to_plot[2][0], 'bo', markersize=4, label='Current Frame')
cursor = [cursor1, cursor2, cursor3]

# Calculate movement boundaries for the view
minMax = np.zeros((2,3))
minMax[0,0] = np.nanmin(markersPosX[3:])
minMax[0,1] = np.nanmin(markersPosY[3:])
minMax[0,2] = np.nanmin(markersPosZ[3:])
minMax[1,0] = np.nanmax(markersPosX[3:])
minMax[1,1] = np.nanmax(markersPosY[3:])
minMax[1,2] = np.nanmax(markersPosZ[3:])
#minMax = np.array([[-2292.197, -2213.251, 15.017],[ 617.451, 760.748, 1815.185]])


# Set appropriate axis limits
ax.set_xlim([minMax[0,0],minMax[1,0]])
ax.set_ylim([minMax[0,1],minMax[1,1]])
ax.set_zlim([minMax[0,2],minMax[1,2]])

# Set the window title
#fig.canvas.manager.window.title("3D Movement")


# Set background white and grid lines white to mimic transparency
plt.rcParams['grid.color'] = 'white'
ax.tick_params(axis='both', colors='white')

# Remove box faces except the ground
ax.xaxis.pane.fill = False
ax.yaxis.pane.fill = False
ax.zaxis.pane.fill = True
#ax.w_zaxis.pane.set_facecolor("w")
ax.xaxis.pane.set_edgecolor("w")
ax.yaxis.pane.set_edgecolor('w')
ax.zaxis.pane.set_edgecolor("w")

# Hide axis lines and ticks
ax.xaxis.line.set_visible(False)
ax.yaxis.line.set_visible(False)
ax.zaxis.line.set_visible(False) 


def update_plot(val):
    #for annotation in ax.texts:
    #    annotation.remove()

    # filter the data based on the current time index
    filteredX = markersPosX[val]
    filteredY = markersPosY[val]
    filteredZ = markersPosZ[val]
    
    # Update scatter plot data instead of creating a new scatter plot
    scatter_plot = ax.collections[0]
    scatter_plot.set_offsets(np.column_stack((filteredX, filteredY)))
    scatter_plot._offsets3d = (filteredX, filteredY, filteredZ)

    # Update bones based on the adjacency matrix
    k = 0
    for i in range(len(adjacencyMatrix)):
        for j in range(0, len(adjacencyMatrix)):
            if adjacencyMatrix[i, j]:

                # Find the existing line corresponding to this edge and update its coordinates
                line = allLines[k]
                line.set_xdata([filteredX[i], filteredX[j]])
                line.set_ydata([filteredY[i], filteredY[j]])
                line.set_3d_properties([filteredZ[i], filteredZ[j]])
                k+=1
    
    # Add labels for each marker
    #for idx, (x, y, z) in enumerate(zip(filteredX, filteredY, filteredZ)):
    #    ax.text(x, y, z, str(idx), color='black', fontsize=14)

    for id, cursor in enumerate([cursor1, cursor2, cursor3]):
        cursor[0].set_xdata([val])
        cursor[0].set_ydata([values_to_plot[id][val]])
    
    fig.canvas.draw_idle()


filteredX = markersPosX[0]
filteredY = markersPosY[0]
filteredZ = markersPosZ[0]

allLines = []

# Plot the first scatters
#print(markersPosX[0].shape, markersPosY[0].shape, markersPosZ[0].shape)
ax.scatter(markersPosX[0],markersPosY[0],markersPosZ[0], s=60, c=colors)
for i in range(len(adjacencyMatrix)):
    for j in range(len(adjacencyMatrix)):
        if adjacencyMatrix[i,j]:
            ax.plot([filteredX[i], filteredX[j]],
                    [filteredY[i], filteredY[j]],
                    [filteredZ[i], filteredZ[j]],
                    color='k', linestyle='-', linewidth=0.7)
            allLines.append(ax.lines[-1])


# Create a slider widget
slider_ax = plt.axes([0.08, 0, 0.8, 0.03])
maxValue = len(markersPosX)-1
slider = Slider(slider_ax, 'Frame:', 0, maxValue, valinit=0, valstep=1)

# Register the update_plot function with the slider widget
slider.on_changed(update_plot)

# Function to update the plot for animation
def animate(frame):
    slider.set_val(frame)

step = 1
anim = animation.FuncAnimation(fig, slider.set_val, frames=range(0, maxValue, step), interval=1, repeat=True, cache_frame_data=False)

last_ = (time.perf_counter(), 0)
remaining_time = [1]

def _print_progress(current_frame, total_frames):
    global last_
    now = time.perf_counter()
    dt = now - last_[0]
    last_frame = last_[1]
    last_ = (now, current_frame)
    if last_frame % 5 == 2:
        remaining_time.append(dt * (total_frames - current_frame))
        if len(remaining_time) > 100:
            remaining_time.pop(0)
        avg_remaining_time = sum(remaining_time) / len(remaining_time)
        print(f"Saving frame {current_frame}/{total_frames} | remaining_time={avg_remaining_time:.4f}s", end="\r")

anim.save(bilateral_dir / f"result_{txt_files[num_file].stem}_{feature_to_plot}.mp4", writer='ffmpeg', fps=30, progress_callback=_print_progress)


# Create a button to pause and play the animation
button_ax = plt.axes([0.92, 0.005, 0.06, 0.03])
button = Button(button_ax, 'Pause/Play')
running = True

# Function to pause and play the animation
def pause_play(event):
    global running 
    if running:
        anim.pause()
    else: 
        anim.resume()
    running ^= True

button.on_clicked(pause_play)
anim.pause()


# Show the plot
plt.show()


qt.qpa.wayland: There are no outputs - creating placeholder screen
